In [1]:
import pandas as pd

In [2]:
item_category_df = pd.read_csv("data/item_categories.csv")
items_df = pd.read_csv("data/items.csv")
shops_df = pd.read_csv("data/shops.csv")

train_df = pd.read_csv("data/sales_train.csv")
test_df = pd.read_csv("data/test.csv")

In [3]:
train_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
date_df = pd.DataFrame(train_df["date"].str.split(".").tolist(),  columns = ['day','month','year'])
date_df.head()

,day,month,year
0,02,01,2013
1,03,01,2013
2,05,01,2013
3,06,01,2013
4,15,01,2013


In [5]:
pre_df = pd.merge(train_df, date_df, left_index=True, right_index=True)
del pre_df["date"]
del pre_df["date_block_num"]
pre_df.head()

,shop_id,item_id,item_price,item_cnt_day,day,month,year
0,59,22154,999.00,1.0,02,01,2013
1,25,2552,899.00,1.0,03,01,2013
2,25,2552,899.00,-1.0,05,01,2013
3,25,2554,1709.05,1.0,06,01,2013
4,25,2555,1099.00,1.0,15,01,2013


In [6]:
monthly_pre_df = pre_df.groupby(["shop_id","item_id", "month"]).mean()

## Merge item_category_with_item_id

In [7]:
del items_df["item_name"]
items_df.head()

,item_id,item_category_id
0,0,40
1,1,76
2,2,40
3,3,40
4,4,40


In [8]:
monthly_pre_df.head()

item_price  item_cnt_day
shop_id item_id month                          
0       30      02          265.0      3.444444
        31      02          434.0      1.571429
        32      01          221.0      1.500000
                02          221.0      1.428571
        33      01          347.0      1.000000

In [9]:
working_df = monthly_pre_df.reset_index().merge(items_df, on="item_id")

In [10]:
working_df.head()

,shop_id,item_id,month,item_price,item_cnt_day,item_category_id
0,0,30,02,265.0,3.444444,40
1,1,30,02,265.0,1.428571,40
2,2,30,03,359.0,1.000000,40
3,2,30,04,169.0,1.000000,40
4,2,30,05,169.0,1.000000,40


In [31]:
# working_df = working_df.merge(pd.get_dummies(working_df["shop_id"], prefix="shop_"), left_index=True, right_index=True)
# working_df = working_df.merge(
#     pd.get_dummies(working_df["item_id"], prefix="item_"), left_index=True, right_index=True)
# working_df = working_df.merge(
#     pd.get_dummies(working_df["categry_id"], prefix="categry_"), left_index=True, right_index=True)
# working_df = working_df.merge(
#     pd.get_dummies(working_df["month"], prefix="month_"), left_index=True, right_index=True)

In [13]:
from sklearn.preprocessing import OneHotEncoder

In [18]:
one_hot_attr = working_df[["shop_id", "item_id", "month", "item_category_id"]]

In [19]:
onehot_enc =OneHotEncoder()

In [36]:
import numpy as np 
one_hot_attr.values.astype(int)

array([[    0,    30,     2,    40],
       [    1,    30,     2,    40],
       [    2,    30,     3,    40],
       ...,
       [   59, 16270,     7,    75],
       [   59, 17732,    10,    28],
       [   59, 18018,     1,    57]])

In [41]:
onehot_enc.fit(one_hot_attr.values.astype(int))

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [ ]:
category_values = onehot_enc.transform(one_hot_attr.values.astype(int))
category_values.shape

In [59]:
category_values

<1396971x21963 sparse matrix of type '<class 'numpy.float64'>'
	with 5587884 stored elements in Compressed Sparse Row format>

In [85]:
from scipy.sparse import csr_matrix
reald_values = working_df["item_price"].values.reshape(-1,1)
print(reald_values.shape)
csr_matrix(reald_values)

(1396971, 1)


<1396971x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1396971 stored elements in Compressed Sparse Row format>

In [86]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
minmax.fit(reald_values)
reald_values = minmax.transform(reald_values)

In [87]:
from scipy.sparse import hstack
total_csc_matrix = hstack([category_values,csr_matrix(reald_values)], format='csr')

In [90]:
y = working_df["item_cnt_day"].values

In [95]:
from sklearn.linear_model import Lasso,LinearRegression
lr_simple = LinearRegression().fit(total_csc_matrix, y)
lr_lasso = Lasso().fit(total_csc_matrix, y)

# Model to testset

In [145]:
test_df.head()

,ID,shop_id,item_id,month
0,0,5,5037,11
1,1,5,5320,11
2,2,5,5233,11
3,3,5,5232,11
4,4,5,5268,11


In [146]:
test_df["month"] = 11

In [147]:
test_df.head()

,ID,shop_id,item_id,month
0,0,5,5037,11
1,1,5,5320,11
2,2,5,5233,11
3,3,5,5232,11
4,4,5,5268,11


In [148]:
test_preprocessing_df = test_df.merge(items_df, on="item_id")
test_preprocessing_df.head()

,ID,shop_id,item_id,month,item_category_id
0,0,5,5037,11,19
1,5100,4,5037,11,19
2,10200,6,5037,11,19
3,15300,3,5037,11,19
4,20400,2,5037,11,19


In [149]:
test_preprocessing_df.shape

(214200, 5)

In [150]:
test_item_price_df = pre_df.groupby(["shop_id","item_id"]).mean().reset_index()

In [151]:
test_item_price_df["temp-id"] = test_item_price_df["shop_id"].map(lambda x : str(x)) + "-" + test_item_price_df["item_id"].map(lambda x : str(x))

In [152]:
test_item_price_df.head()
del test_item_price_df["item_cnt_day"]

In [153]:
test_preprocessing_df["temp-id"] = test_preprocessing_df["shop_id"].map(lambda x : str(x)) + "-" + test_preprocessing_df["item_id"].map(lambda x : str(x))
test_preprocessing_df.head()
test_preprocessing_df.shape

(214200, 6)

In [154]:
test_preprocessing_df = test_preprocessing_df.merge(
    test_item_price_df[["item_price", "temp-id"]], on="temp-id", how="left")
test_preprocessing_df.head()

,ID,shop_id,item_id,month,item_category_id,temp-id,item_price
0,0,5,5037,11,19,5-5037,1633.692308
1,5100,4,5037,11,19,4-5037,1836.583333
2,10200,6,5037,11,19,6-5037,2213.705882
3,15300,3,5037,11,19,3-5037,2241.105263
4,20400,2,5037,11,19,2-5037,1857.375000


In [174]:
test_preprocessing_df.loc[test_preprocessing_df["item_price"].isnull(), "item_price"] = test_preprocessing_df["item_price"].mean()

In [176]:
test_preprocessing_df.isnull().sum()

ID                  0
shop_id             0
item_id             0
month               0
item_category_id    0
temp-id             0
item_price          0
dtype: int64

In [177]:
test_category = test_preprocessing_df[["shop_id", "item_id", "month", "item_category_id"]].values

In [181]:
test_preprocessing_df = test_preprocessing_df.sort_values("ID")

In [182]:
test_category_values = onehot_enc.transform(test_category)
print(test_category_values.shape)
test_category_values

(214200, 21963)


<214200x21963 sparse matrix of type '<class 'numpy.float64'>'
	with 841554 stored elements in Compressed Sparse Row format>

In [190]:
test_real_values = minmax.transform(test_preprocessing_df["item_price"].values.reshape(-1,1))

In [192]:
from scipy.sparse import hstack
total_test_csc_matrix = hstack([test_category_values,csr_matrix(test_real_values)], format='csr')

In [193]:
total_test_csc_matrix.shape

(214200, 21964)

In [199]:
y_hat_simple = lr_simple.predict(total_test_csc_matrix)

In [209]:
simple_lr_result = pd.DataFrame(y_hat_simple, columns=["item_cnt_month"])
simple_lr_result.index.name = 'ID'
simple_lr_result.to_csv("simple_lr_result.csv", sep=",")
simple_lr_result.head()

,item_cnt_month
ID,
0,1.075285
1,1.090275
2,1.134832
3,1.075095
4,1.094642


In [ ]:
y_hat_lasso = lr_lasso.predict(total_test_csc_matrix)

In [210]:
lasso_lr_result = pd.DataFrame(y_hat_lasso, columns=["item_cnt_month"])
lasso_lr_result.index.name = 'ID'
lasso_lr_result.to_csv("lasso_lr_result.csv", sep=",")
lasso_lr_result.head()

,item_cnt_month
ID,
0,1.067837
1,1.067837
2,1.067837
3,1.067837
4,1.067837
